<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/gemini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
#pip install -q -U google-generativeai

In [45]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [46]:
import pandas as pd

In [47]:
model = genai.GenerativeModel('gemini-pro')

In [48]:
def detect(body):
    try:
        #specify prompt
        prompt =f"SMS Body: {body}\nIs this message real or fake?"
        # apply prompt to model and store in response object
        response= model.generate_content(prompt)

        #check if response is real or fake
        if "real" in response.text.lower():
            return "Real"
        elif "fake" in response.text.lower():
            return "Fake"
        else:
            return "Undetermined"

    except Exception as error:
        print("Error occured:", str(error))
        return "Error"

In [49]:
from google.colab import files
uploaded = files.upload()

Saving smishTankSet.xlsx to smishTankSet (5).xlsx


In [50]:
#process excel file
def process_excel(inputFile):
  #assign file to pandas dataframe
  df=pd.read_excel(inputFile, header=None)
  #rename columns
  df.columns = ['SMS Body']

  # to store results at each value
  authenticityResults = []

  # process each row, apply detect function and print results
  for index, row in df.iterrows():
      #apply detect function on each row
      authResult = detect(row['SMS Body'])
      authenticityResults.append(authResult)

      #print status after each orw
      print(f"Processed input {index + 1}, Result = {authResult}")

  #assign results to new column
  df['Authenticity'] = authenticityResults

  #write updated dataframe to new file
  df.to_excel("processed_" + inputFile, index=False)


In [51]:
#process file
pFile = next(iter(uploaded))
process_excel(pFile)

Processed input 1, Result = Fake
Processed input 2, Result = Fake
Processed input 3, Result = Fake
Processed input 4, Result = Real
Processed input 5, Result = Real
Processed input 6, Result = Fake
Processed input 7, Result = Fake
Processed input 8, Result = Fake
Processed input 9, Result = Real
Processed input 10, Result = Real
Processed input 11, Result = Fake
Processed input 12, Result = Undetermined
Processed input 13, Result = Fake
Processed input 14, Result = Fake
Processed input 15, Result = Fake
Processed input 16, Result = Real
Processed input 17, Result = Fake
Processed input 18, Result = Fake
Processed input 19, Result = Fake
Processed input 20, Result = Fake
Processed input 21, Result = Fake
Processed input 22, Result = Fake
Processed input 23, Result = Fake
Processed input 24, Result = Fake
Processed input 25, Result = Fake
Processed input 26, Result = Fake
Processed input 27, Result = Fake
Processed input 28, Result = Fake
Processed input 29, Result = Fake
Processed input

In [52]:
#download modified file to local machine
files.download("processed_" + pFile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>